## <span style="color:red"> Run data generation with DARTS</span>

In [ ]:
import pandas as pd
from darts.hdata.model import Model

In [ ]:
# user introduced wells
new_well_list = ['P04', 'P05', 'P06', 'P09', 'I10', 'P110']
new_well_x    = [   17,    113,   88,   144,   87,   65   ]
new_well_y    = [   18,    17,   83,   170,   45,   17   ]
n_new_wells = len(new_well_list)

# define the model
m = Model()

# adding new wells to the model
for i in range(n_new_wells):
    m.add_well(new_well_list[i], new_well_x[i], new_well_y[i])
        
m.init()  # initiate the model
m.run_python(m.params.first_ts)  # initial run of DARTS

# Define you infromation and strategy below:

group_name = 'ERCE'

# define controls for all wells
for i, w in enumerate(m.reservoir.wells):
    if 'I02' in w.name:
        m.set_well_control(w, +315 * 24) # rate in m3/day
    elif 'P02' in w.name:
        m.set_well_control(w, -315 * 24) # rate in m3/day
    # Operate E03 as I04
    elif 'E03' in w.name:
        m.set_well_control(w, +700 * 24) # rate in m3/day
    elif 'P04' in w.name:
        m.set_well_control(w, -700 * 24) # rate in m3/day
    # Operate E02 as I05
    elif 'E02' in w.name:
        m.set_well_control(w, +350 * 24) # rate in m3/day
    # Operate E04 as I06
    elif 'E04' in w.name:
        m.set_well_control(w, +500 * 24) # rate in m3/day
    elif 'P06' in w.name:
        m.set_well_control(w, -500 * 24) # rate in m3/day
    # Operate E06 as I09
    elif 'E06' in w.name:
        m.set_well_control(w, -350 * 24) # rate in m3/day
    elif 'I09' in w.name:
        m.set_well_control(w, +240 * 24) # rate in m3/day
    elif 'I10' in w.name:
        m.set_well_control(w, +450 * 24) # rate in m3/day
    elif 'P10' in w.name:
        m.set_well_control(w, -450 * 24) # rate in m3/day
    # Injectors
    elif 'I' in w.name:
        m.set_well_control(w, +350 * 24) # rate in m3/day
    # Producers
    else:
        m.set_well_control(w, -350 * 24) # rate in m3/day

m.run_python(365, restart_dt=1.36e-6)  # run the first control for 1 year
m.run_python(49*365)  # ... and the rest for 50 years

In [ ]:
time_data = pd.DataFrame.from_dict(m.physics.engine.time_data)
writer = pd.ExcelWriter(group_name + '_time_data.xlsx')
time_data.to_excel(writer, 'Sheet1')
writer.save()